In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import keras
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input,InputLayer, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras import applications, initializers, Model, optimizers, metrics
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score, mean_absolute_percentage_error, f1_score
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, LabelEncoder
from tensorflow.keras.utils import to_categorical

from keras.callbacks import LearningRateScheduler, ModelCheckpoint

In [2]:
data = pd.read_csv("datosnuevos.csv",encoding='latin-1',index_col=0,sep = ";")
data = data.dropna()

data = data.rename(columns={'P47,':'P47'})
data['P47'] = data['P47'].str.replace(",","")

data = data.drop(['Perfil','Periodo','Curso'], axis=1)


In [3]:
encode_norm = {"Totalmente en desacuerdo":-0.9,"Levemente en desacuerdo":-0.4,"Ni de deacuerdo ni en desacuerdo":-0.1,"Medianamente de acuerdo":0.4,"Totalmente de acuerdo":0.9}
encode = {"Totalmente en desacuerdo":'0',"Levemente en desacuerdo":'1',"Ni de deacuerdo ni en desacuerdo":'2',"Medianamente de acuerdo":'3',"Totalmente de acuerdo":'4'}

In [4]:
preguntas = []
for i in range(1,48):
    preguntas.append("P"+str(i))
    data["P"+str(i)] = data["P"+str(i)].map(encode_norm)
    #data["P"+str(i)] = data["P"+str(i)].astype("category")

In [5]:
preguntas_adiccion = ['P2','P7','P9','P11','P12','P13','P15',
'P17','P23','P25','P26','P27','P32','P34','P38','P40','P41','P42','P43',
'P44','P45','P46','P47']

In [6]:
X = data.loc[:,preguntas]
y = data.loc[:,preguntas_adiccion]
X = X.drop(preguntas_adiccion, axis = 1)

In [7]:
norm = MinMaxScaler()
for p in preguntas_adiccion:
    y[p] = y[p].map({-0.9:0,-0.4:1,-0.1:2,0.4:3,0.9:4})
y = y.apply(pd.to_numeric)
y = y.sum(axis=1)
#y = norm.fit_transform()

In [8]:
X = X.apply(pd.to_numeric)
preg = X.sum(axis = 1)

In [9]:
y = (y)/(92)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size = 0.5, random_state=1)

In [11]:
n_rep = 50

In [12]:
m1 = 0
r1 = 0
for i in range(0,n_rep):
    input_shape = (X_train.shape[1],)
    model = Sequential()
    #Input
    model.add(InputLayer(input_shape=input_shape))

    #Hidden
    model.add(Dense(4, activation = "tanh", use_bias = True))
    #Output
    model.add(Dense(1, activation = 'linear'))


    model.compile(loss="mean_squared_error",                
                optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),  
                metrics=['accuracy'])

    callback_es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1)
    annealer = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** x)

    model.fit(X_train, y_train, validation_data=(X_val,y_val), callbacks=[annealer, callback_es], batch_size = 8, epochs=50)
    probs = model.predict(X_test)
    m1 += mean_squared_error(y_test,probs)
    r1 += r2_score(y_test,probs)
m1 = m1/n_rep
r1 = r1/n_rep

Epoch 1/50
257/257 [==============================] - 1s 1ms/step - loss: 0.0941 - accuracy: 0.0273 - val_loss: 0.0548 - val_accuracy: 0.0114
Epoch 2/50
257/257 [==============================] - 0s 869us/step - loss: 0.0343 - accuracy: 0.0322 - val_loss: 0.0242 - val_accuracy: 0.0205
Epoch 3/50
257/257 [==============================] - 0s 822us/step - loss: 0.0173 - accuracy: 0.0336 - val_loss: 0.0132 - val_accuracy: 0.0205
Epoch 4/50
257/257 [==============================] - 0s 708us/step - loss: 0.0113 - accuracy: 0.0336 - val_loss: 0.0095 - val_accuracy: 0.0205
Epoch 5/50
257/257 [==============================] - 0s 818us/step - loss: 0.0091 - accuracy: 0.0336 - val_loss: 0.0080 - val_accuracy: 0.0205
Epoch 6/50
257/257 [==============================] - 0s 1ms/step - loss: 0.0081 - accuracy: 0.0336 - val_loss: 0.0074 - val_accuracy: 0.0205
Epoch 7/50
257/257 [==============================] - 0s 793us/step - loss: 0.0077 - accuracy: 0.0336 - val_loss: 0.0071 - val_accuracy: 0.0

In [13]:
m2 = 0
r2 = 0
for i in range(0,n_rep):
    input_shape = (X_train.shape[1],)
    model = Sequential()
    #Input
    model.add(InputLayer(input_shape=input_shape))

    #Hidden
    model.add(Dense(6, activation = "tanh", use_bias = True))
    #Output
    model.add(Dense(1, activation = 'linear'))


    model.compile(loss="mean_squared_error",                
                optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),  
                metrics=['accuracy'])

    callback_es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1)
    annealer = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** x)
    
    model.fit(X_train, y_train, validation_data=(X_val,y_val), callbacks=[annealer, callback_es], batch_size = 8, epochs=50)
    probs = model.predict(X_test)
    m2 += mean_squared_error(y_test,probs)
    r2 += r2_score(y_test,probs)
m2 = m2/n_rep
r2 = r2/n_rep

Epoch 1/50
257/257 [==============================] - 0s 991us/step - loss: 0.1240 - accuracy: 0.0268 - val_loss: 0.0479 - val_accuracy: 0.0114
Epoch 2/50
257/257 [==============================] - 0s 627us/step - loss: 0.0321 - accuracy: 0.0278 - val_loss: 0.0233 - val_accuracy: 0.0136
Epoch 3/50
257/257 [==============================] - 0s 625us/step - loss: 0.0169 - accuracy: 0.0293 - val_loss: 0.0154 - val_accuracy: 0.0205
Epoch 4/50
257/257 [==============================] - 0s 651us/step - loss: 0.0124 - accuracy: 0.0336 - val_loss: 0.0127 - val_accuracy: 0.0205
Epoch 5/50
257/257 [==============================] - 0s 615us/step - loss: 0.0105 - accuracy: 0.0336 - val_loss: 0.0113 - val_accuracy: 0.0205
Epoch 6/50
257/257 [==============================] - 0s 633us/step - loss: 0.0096 - accuracy: 0.0336 - val_loss: 0.0103 - val_accuracy: 0.0205
Epoch 7/50
257/257 [==============================] - 0s 611us/step - loss: 0.0089 - accuracy: 0.0336 - val_loss: 0.0098 - val_accuracy:

In [14]:
m3 = 0
r3 = 0
for i in range(0,n_rep):
    input_shape = (X_train.shape[1],)
    model = Sequential()
    #Input
    model.add(InputLayer(input_shape=input_shape))

    #Hidden
    model.add(Dense(1500, activation = "relu"))
    model.add(Dense(1500, activation = "relu"))
    #Output
    model.add(Dense(1, activation = 'softmax'))


    model.compile(loss="mean_squared_error",                
                optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),  
                metrics=['accuracy'])

    callback_es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1)
    annealer = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** x)

    model.fit(X_train, y_train, validation_data=(X_val,y_val), callbacks=[annealer, callback_es], batch_size = 8, epochs=50)
    probs = model.predict(X_test)
    m3 += mean_squared_error(y_test,probs)
    r3 += r2_score(y_test,probs)
m3 = m3/n_rep
r3 = r3/n_rep

Epoch 1/50
257/257 [==============================] - 4s 13ms/step - loss: 0.6215 - accuracy: 0.0063 - val_loss: 0.6143 - val_accuracy: 0.0091
Epoch 2/50
257/257 [==============================] - 3s 12ms/step - loss: 0.6215 - accuracy: 0.0063 - val_loss: 0.6143 - val_accuracy: 0.0091
Epoch 1/50
257/257 [==============================] - 4s 13ms/step - loss: 0.6215 - accuracy: 0.0063 - val_loss: 0.6143 - val_accuracy: 0.0091
Epoch 2/50
257/257 [==============================] - 3s 12ms/step - loss: 0.6215 - accuracy: 0.0063 - val_loss: 0.6143 - val_accuracy: 0.0091
Epoch 1/50
257/257 [==============================] - 4s 13ms/step - loss: 0.6215 - accuracy: 0.0063 - val_loss: 0.6143 - val_accuracy: 0.0091
Epoch 2/50
257/257 [==============================] - 3s 13ms/step - loss: 0.6215 - accuracy: 0.0063 - val_loss: 0.6143 - val_accuracy: 0.0091
Epoch 1/50
257/257 [==============================] - 4s 13ms/step - loss: 0.6215 - accuracy: 0.0063 - val_loss: 0.6143 - val_accuracy: 0.0091

In [15]:
m4 = 0
r4 = 0
for i in range(0,n_rep):
    input_shape = (X_train.shape[1],)
    model = Sequential()
    #Input
    model.add(InputLayer(input_shape=input_shape))

    #Hidden
    model.add(Dense(10, activation = "tanh"))
    #Output
    model.add(Dense(1, activation = 'linear'))


    model.compile(loss="mean_squared_error",                
                optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),  
                metrics=['accuracy'])

    callback_es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1)
    annealer = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** x)

    model.fit(X_train, y_train, validation_data=(X_val,y_val), callbacks=[annealer, callback_es], batch_size = 8, epochs=50)
    probs = model.predict(X_test)
    m4 += mean_squared_error(y_test,probs)
    r4 += r2_score(y_test,probs)
m4 = m4/n_rep
r4 = r4/n_rep

Epoch 1/50
257/257 [==============================] - 0s 968us/step - loss: 0.2144 - accuracy: 0.0254 - val_loss: 0.0683 - val_accuracy: 0.0114
Epoch 2/50
257/257 [==============================] - 0s 643us/step - loss: 0.0405 - accuracy: 0.0297 - val_loss: 0.0269 - val_accuracy: 0.0136
Epoch 3/50
257/257 [==============================] - 0s 621us/step - loss: 0.0217 - accuracy: 0.0336 - val_loss: 0.0200 - val_accuracy: 0.0205
Epoch 4/50
257/257 [==============================] - 0s 619us/step - loss: 0.0169 - accuracy: 0.0336 - val_loss: 0.0165 - val_accuracy: 0.0205
Epoch 5/50
257/257 [==============================] - 0s 628us/step - loss: 0.0145 - accuracy: 0.0336 - val_loss: 0.0145 - val_accuracy: 0.0205
Epoch 6/50
257/257 [==============================] - 0s 617us/step - loss: 0.0129 - accuracy: 0.0336 - val_loss: 0.0126 - val_accuracy: 0.0205
Epoch 7/50
257/257 [==============================] - 0s 638us/step - loss: 0.0119 - accuracy: 0.0336 - val_loss: 0.0120 - val_accuracy:

In [16]:
m5 = 0
r5 = 0
for i in range(0,n_rep):
    input_shape = (X_train.shape[1],)
    model = Sequential()
    #Input
    model.add(InputLayer(input_shape=input_shape))

    #Hidden
    model.add(Dense(16, activation = "tanh"))
    model.add(Dense(16, activation = "tanh"))
    model.add(Dense(16, activation = "tanh"))
    #Output
    model.add(Dense(1, activation = 'linear'))


    model.compile(loss="mean_squared_error",                
                optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),  
                metrics=['accuracy'])

    callback_es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1)
    annealer = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** x)

    model.fit(X_train, y_train, validation_data=(X_val,y_val), callbacks=[annealer, callback_es], batch_size = 8, epochs=50)
    probs = model.predict(X_test)
    m5 += mean_squared_error(y_test,probs)
    r5 += r2_score(y_test,probs)
m5 = m5/n_rep
r5 = r5/n_rep

Epoch 1/50
257/257 [==============================] - 1s 1ms/step - loss: 0.0291 - accuracy: 0.0302 - val_loss: 0.0107 - val_accuracy: 0.0205
Epoch 2/50
257/257 [==============================] - 0s 865us/step - loss: 0.0087 - accuracy: 0.0336 - val_loss: 0.0084 - val_accuracy: 0.0205
Epoch 3/50
257/257 [==============================] - 0s 906us/step - loss: 0.0077 - accuracy: 0.0336 - val_loss: 0.0092 - val_accuracy: 0.0205
Epoch 1/50
257/257 [==============================] - 1s 1ms/step - loss: 0.0242 - accuracy: 0.0327 - val_loss: 0.0081 - val_accuracy: 0.0205
Epoch 2/50
257/257 [==============================] - 0s 791us/step - loss: 0.0088 - accuracy: 0.0336 - val_loss: 0.0077 - val_accuracy: 0.0205
Epoch 3/50
257/257 [==============================] - 0s 825us/step - loss: 0.0080 - accuracy: 0.0336 - val_loss: 0.0077 - val_accuracy: 0.0205
Epoch 4/50
257/257 [==============================] - 0s 821us/step - loss: 0.0075 - accuracy: 0.0336 - val_loss: 0.0076 - val_accuracy: 0.0

In [17]:
pd.DataFrame(data = [[m1,m2,m3,m4,m5],[r1,r2,r3,r4,r5]], columns = ['Moral-Cuadra et al.', 'Solano-Sanchez et al.','Lin et al.', 'Petek et al.', 'Hong et al.'],index = ['MSE', 'R2'])

,Moral-Cuadra et al.,Solano-Sanchez et al.,Lin et al.,Petek et al.,Hong et al.
MSE,0.007375,0.007952,0.624940,0.008214,0.007501
R2,0.814690,0.800202,-14.702894,0.793609,0.811510


In [18]:
print("MSE: ",m1)
print("R2: ",r1)

MSE:  0.00737491348125923
R2:  0.8146901106742984


In [19]:
df = pd.DataFrame(columns = ['MSE', 'R2', 'n', 'l'])
for j in range(0,6):
    m1 = 0
    r1 = 0

    neurons = 0
    if j%2 == 0:
        neurons = 8
    else:
        neurons = 1024
    
    if int(j/2) == 0:
        n_layers = 1
    elif int(j/2) == 1:
        n_layers = 2
    else:
        n_layers =3

    for i in range(0,n_rep):

        input_shape = (X_train.shape[1],)
        model = Sequential()
        #Input
        model.add(InputLayer(input_shape=input_shape))

        #Hidden
        for i in range(n_layers):
            model.add(Dense(neurons, activation = "relu"))
        #Output
        model.add(Dense(1, activation = 'sigmoid'))


        model.compile(loss="mean_squared_error",                
                    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),  
                    metrics=['accuracy'])

        callback_es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1)
        annealer = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** x)

        model.fit(X_train, y_train, validation_data=(X_val,y_val), callbacks=[annealer, callback_es], batch_size = 8, epochs=50, verbose = 0)
        probs = model.predict(X_test)
        m1 += mean_squared_error(y_test,probs)
        r1 += r2_score(y_test,probs)
    m1 = m1/n_rep
    r1 = r1/n_rep
    df = df.append({'MSE':m1, 'R2': r1, 'n':neurons, 'l':n_layers}, ignore_index=True)
df

C:\Users\JoseA\AppData\Local\Temp\ipykernel_9692\1648138987.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'MSE':m1, 'R2': r1, 'n':neurons, 'l':n_layers}, ignore_index=True)
C:\Users\JoseA\AppData\Local\Temp\ipykernel_9692\1648138987.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'MSE':m1, 'R2': r1, 'n':neurons, 'l':n_layers}, ignore_index=True)
C:\Users\JoseA\AppData\Local\Temp\ipykernel_9692\1648138987.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'MSE':m1, 'R2': r1, 'n':neurons, 'l':n_layers}, ignore_index=True)
C:\Users\JoseA\AppData\Local\Temp\ipykernel_9692\1648138987.py:46: FutureWarning: The frame.append method is deprecated and will be removed from 

,MSE,R2,n,l
0,0.006482,0.837128,8.0,1.0
1,0.0063,0.841696,1024.0,1.0
2,0.006411,0.838914,8.0,2.0
3,0.006494,0.836834,1024.0,2.0
4,0.006477,0.83724,8.0,3.0
5,0.006965,0.824981,1024.0,3.0
